In [2]:
import numpy as np

import pandas

features = pandas.read_csv('features.csv', index_col='match_id')
features_test = pandas.read_csv('features_test.csv')
features.head()
#features_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


Выведем данные и уберем колонки-спойлеры. Также выделим таргет-столбец.

In [3]:
signs = features.drop(columns = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'])
print(signs)
target = features['radiant_win']

          start_time  lobby_type  r1_hero  r1_level  r1_xp  r1_gold  r1_lh  \
match_id                                                                     
0         1430198770           7       11         5   2098     1489     20   
1         1430220345           0       42         4   1188     1033      9   
2         1430227081           7       33         4   1319     1270     22   
3         1430263531           1       29         4   1779     1056     14   
4         1430282290           7       13         4   1431     1090      8   
...              ...         ...      ...       ...    ...      ...    ...   
114402    1450265551           1       47         4   1706     1198     17   
114403    1450277704           0       43         4   1793     1416     17   
114404    1450291848           1       98         4   1399      540      1   
114405    1450292986           1      100         3   1135      766      6   
114406    1450313370           7       50         3   1053      

In [4]:
nan_columns = []
for i in range(102):
    if (signs.count()[i] != 97230):
        nan_columns.append(list(signs.iloc[:, i:i+1].columns)[0])
print(nan_columns)

['first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time']


Собственно, если поля пустые, то за первые пять минут событий, которые учитываются в этих полях, не произошло.

In [5]:
signs = signs.fillna(0)
print(signs)

          start_time  lobby_type  r1_hero  r1_level  r1_xp  r1_gold  r1_lh  \
match_id                                                                     
0         1430198770           7       11         5   2098     1489     20   
1         1430220345           0       42         4   1188     1033      9   
2         1430227081           7       33         4   1319     1270     22   
3         1430263531           1       29         4   1779     1056     14   
4         1430282290           7       13         4   1431     1090      8   
...              ...         ...      ...       ...    ...      ...    ...   
114402    1450265551           1       47         4   1706     1198     17   
114403    1450277704           0       43         4   1793     1416     17   
114404    1450291848           1       98         4   1399      540      1   
114405    1450292986           1      100         3   1135      766      6   
114406    1450313370           7       50         3   1053      

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

In [7]:
from sklearn.model_selection import KFold, cross_val_score

In [8]:
kf = KFold(shuffle = True, random_state = 42)
X = np.array(signs)
y = np.array(target)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [9]:
from pandas.core.common import flatten
from sklearn.metrics import roc_auc_score

In [10]:
import time
import datetime

In [11]:
normalized_signs=(signs-signs.min())/(signs.max()-signs.min())
print(normalized_signs)

          start_time  lobby_type   r1_hero  r1_level     r1_xp   r1_gold  \
match_id                                                                   
0           0.000000    1.000000  0.090090  0.833333  0.632118  0.343721   
1           0.001073    0.000000  0.369369  0.666667  0.357939  0.238458   
2           0.001407    1.000000  0.288288  0.666667  0.397409  0.293167   
3           0.003220    0.142857  0.252252  0.666667  0.536005  0.243767   
4           0.004152    1.000000  0.108108  0.666667  0.431154  0.251616   
...              ...         ...       ...       ...       ...       ...   
114402      0.997623    0.142857  0.414414  0.666667  0.514010  0.276547   
114403      0.998227    0.000000  0.378378  0.666667  0.540223  0.326870   
114404      0.998930    0.142857  0.873874  0.666667  0.421513  0.124654   
114405      0.998987    0.142857  0.891892  0.500000  0.341970  0.176824   
114406      1.000000    1.000000  0.441441  0.500000  0.317264  0.184441   

           

Здесь работаем с градиентным бустингом над решающими деревьями.

In [13]:
for i in range(10, 40, 10):
    print(i)
    start_time = datetime.datetime.now()
    dtc = GradientBoostingClassifier(n_estimators=i, verbose=True, random_state=241, learning_rate = 0.2)
    start_time = datetime.datetime.now()
    r = cross_val_score(dtc, X, y, scoring = 'accuracy', cv = kf)
    print(r.mean())
    if i == 30:
        print('Time elapsed:', datetime.datetime.now() - start_time)
    dtc.fit(X_train, y_train)
    y_pred = dtc.predict(X_test)
    print(y_pred)
    print(roc_auc_score(y_test, y_pred))

10
      Iter       Train Loss   Remaining Time 
         1           1.3728           13.50s
         2           1.3632           11.26s
         3           1.3535            9.56s
         4           1.3446            7.99s
         5           1.3372            6.57s
         6           1.3302            5.84s
         7           1.3236            4.47s
         8           1.3176            2.93s
         9           1.3123            1.45s
        10           1.3072            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3730           13.32s
         2           1.3636           10.88s
         3           1.3537            9.42s
         4           1.3459            9.08s
         5           1.3381            7.61s
         6           1.3309            5.90s
         7           1.3242            4.32s
         8           1.3183            3.00s
         9           1.3127            1.53s
        10           1.3076            0.00s
     

         3           1.3538            1.00m
         4           1.3449           57.69s
         5           1.3373           55.96s
         6           1.3304           53.73s
         7           1.3245           51.48s
         8           1.3184           49.33s
         9           1.3128           48.20s
        10           1.3078           46.49s
        20           1.2736           23.76s
        30           1.2543            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3726            1.23m
         2           1.3629            1.15m
         3           1.3535            1.08m
         4           1.3450            1.06m
         5           1.3376            1.01m
         6           1.3307           59.40s
         7           1.3244           57.71s
         8           1.3184           55.41s
         9           1.3127           52.73s
        10           1.3077           50.04s
        20           1.2736           23.88s
        3

Время получилось 2.11, а лучшая точность около 0.64. Нот грейт, нот террибл...
Больше деревьев сделать можно, насколько мне помнится, в том и прелесть метода, а для ускорения процесса можно уменьшить максимальную глубину или взять подвыборку.

А тут начинается логистическая регрессия

In [17]:
kf = KFold(shuffle = True, random_state = 42)
X = np.array(normalized_signs)
y = np.array(target)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [18]:
from sklearn.linear_model import LogisticRegression
for i in range(1, 100, 10):
    print(i)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C = i, max_iter = 300)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
#print(predictions)
    print(roc_auc_score(y_test, predictions))
    print('Time elapsed:', datetime.datetime.now() - start_time)

1
0.6596402787871496
Time elapsed: 0:00:05.231727
11
0.6590679915726628
Time elapsed: 0:00:05.092578
21
0.6590145556423432
Time elapsed: 0:00:04.916078
31
0.6589115607864613
Time elapsed: 0:00:05.133659
41
0.6592205453541073
Time elapsed: 0:00:05.317137
51
0.6591709864285449
Time elapsed: 0:00:05.161737
61
0.6589611197120238
Time elapsed: 0:00:04.832852
71
0.6588620018608988
Time elapsed: 0:00:04.639337
81
0.6590641145679058
Time elapsed: 0:00:04.800819
91
0.6590679915726628
Time elapsed: 0:00:04.587159


Качество чуточку повыше и работает метод значительно быстрее. А ведь мы еще и категориальные признаки не трогали...
С округлением метрика везде порядка 0.66.

In [19]:
droppin = normalized_signs.drop(columns = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])
print(droppin)

          start_time  r1_level     r1_xp   r1_gold     r1_lh  r1_kills  \
match_id                                                                 
0           0.000000  0.833333  0.632118  0.343721  0.425532     0.000   
1           0.001073  0.666667  0.357939  0.238458  0.191489     0.000   
2           0.001407  0.666667  0.397409  0.293167  0.468085     0.000   
3           0.003220  0.666667  0.536005  0.243767  0.297872     0.000   
4           0.004152  0.666667  0.431154  0.251616  0.170213     0.125   
...              ...       ...       ...       ...       ...       ...   
114402      0.997623  0.666667  0.514010  0.276547  0.361702     0.000   
114403      0.998227  0.666667  0.540223  0.326870  0.361702     0.000   
114404      0.998930  0.666667  0.421513  0.124654  0.021277     0.000   
114405      0.998987  0.500000  0.341970  0.176824  0.127660     0.000   
114406      1.000000  0.500000  0.317264  0.184441  0.148936     0.000   

          r1_deaths  r1_items  r2_lev

In [20]:
kf = KFold(shuffle = True, random_state = 42)
X = np.array(droppin)
y = np.array(target)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [55]:
for i in range(1, 100, 10):
    print(i)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C = i, max_iter = 300)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
#print(predictions)
    print(roc_auc_score(y_test, predictions))
    print('Time elapsed:', datetime.datetime.now() - start_time)

1
0.6599221243214964
Time elapsed: 0:00:03.710374
11
0.6599833142613301
Time elapsed: 0:00:03.904437
21
0.6598725653959339
Time elapsed: 0:00:04.234525
31
0.6597656935352948
Time elapsed: 0:00:03.672849
41
0.6597161346097323
Time elapsed: 0:00:03.976692
51
0.6598725653959339
Time elapsed: 0:00:03.896440
61
0.6597695705400518
Time elapsed: 0:00:03.815731
71
0.6598686883911768
Time elapsed: 0:00:04.100054
81
0.6597656935352948
Time elapsed: 0:00:04.314533
91
0.6598268834751283
Time elapsed: 0:00:04.692980


Если категориальные признаки просто убрать, то сильно лучше не становится, но мы все же чуть-чуть в выигрыше.

In [21]:
features['r1_hero'].value_counts()

11     5045
7      3899
72     3551
39     3341
112    3119
       ... 
66      108
80       88
103      75
58       69
109      20
Name: r1_hero, Length: 108, dtype: int64

Разных героев у нас 108, но что интересно, далее метод мешка слов требует чуть больше элементов. Я так и не понял, почему, буду рад объяснениям.

In [23]:
X_pick = np.zeros((signs.shape[0], 112))

for i, match_id in enumerate(signs.index):
    for p in range(5):
        X_pick[i, signs.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, signs.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
print(X_pick)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0. -1.]]


In [24]:
inter = np.concatenate((X.transpose(), X_pick.transpose()))
X_new = inter.transpose()

In [25]:
for train_index, test_index in kf.split(X_new):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [26]:
for i in range(1, 100, 10):
    print(i/10)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', C = i/10, max_iter = 500)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
#print(predictions)
    print(roc_auc_score(y_test, predictions))
    print('Time elapsed:', datetime.datetime.now() - start_time)

0.1
0.6592808931576619
Time elapsed: 0:00:02.101226
1.1
0.6599833142613301
Time elapsed: 0:00:03.212512
2.1
0.6597238886192465
Time elapsed: 0:00:02.855584
3.1
0.6599337553357675
Time elapsed: 0:00:04.332524
4.1
0.6600367501916495
Time elapsed: 0:00:03.760001
5.1
0.6599337553357675
Time elapsed: 0:00:04.115984
6.1
0.6599298783310105
Time elapsed: 0:00:03.717890
7.1
0.6598803194054479
Time elapsed: 0:00:04.123970
8.1
0.6599833142613301
Time elapsed: 0:00:04.285396
9.1
0.6598268834751283
Time elapsed: 0:00:04.406721


Примерно все о том же.

In [27]:
clf = LogisticRegression(penalty='l2', C = 1, max_iter = 300)
clf.fit(X_new, target)
print(clf.predict_proba(X_new))

[[0.33683157 0.66316843]
 [0.26419029 0.73580971]
 [0.39522467 0.60477533]
 ...
 [0.54157225 0.45842775]
 [0.58279998 0.41720002]
 [0.06659305 0.93340695]]


Тут оптимальное С взял. Как финальный алгоритм, я выбрал метод логистической регрессии, он и быстрее, и чуть-чуть точнее.

Подготовим тестовые данные и получим прогноз. К сожалению, соревнование на кеггле по этим данным вроде как закрылось, так что проверить, насколько все хорошо или плохо, возможным не представилось.

In [29]:
features_test = pandas.read_csv('features_test.csv', index_col='match_id')
print(features_test)

          start_time  lobby_type  r1_hero  r1_level  r1_xp  r1_gold  r1_lh  \
match_id                                                                     
6         1430287923           0       93         4   1103     1089      8   
7         1430293357           1       20         2    556      570      1   
10        1430301774           1      112         2    751      808      1   
13        1430323933           1       27         3    708      903      1   
16        1430331112           1       39         4   1259      661      4   
...              ...         ...      ...       ...    ...      ...    ...   
114369    1450212780           7       11         5   2054     1941     27   
114377    1450222875           1        3         3    748      605      1   
114378    1450223593           1       85         2    575      499      0   
114393    1450244771           0        7         4   1844     1176      8   
114398    1450255429           1        8         4   1215     1

In [30]:
signs_test = features_test.fillna(0)
normalized_signs_test=(signs_test-signs_test.min())/(signs_test.max()-signs_test.min())
droppin_test = normalized_signs_test.drop(columns = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])
X_test = np.array(droppin_test)
print(X_test)

[[0.00000000e+00 6.66666667e-01 3.65473824e-01 ... 2.85714286e-01
  0.00000000e+00 5.24804178e-01]
 [2.72142149e-04 3.33333333e-01 1.84227966e-01 ... 2.85714286e-01
  1.66666667e-01 2.58485640e-01]
 [6.93677017e-04 3.33333333e-01 2.48840292e-01 ... 4.28571429e-01
  0.00000000e+00 1.27937337e-01]
 ...
 [9.98405610e-01 3.33333333e-01 1.90523526e-01 ... 5.71428571e-01
  0.00000000e+00 1.22715405e-01]
 [9.99466233e-01 6.66666667e-01 6.11000663e-01 ... 1.42857143e-01
  0.00000000e+00 2.16710183e-01]
 [1.00000000e+00 6.66666667e-01 4.02584493e-01 ... 2.85714286e-01
  0.00000000e+00 2.87206266e-01]]


In [31]:
X_pick_test = np.zeros((signs_test.shape[0], 112))

for i, match_id in enumerate(signs_test.index):
    for p in range(5):
        X_pick_test[i, signs_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, signs_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
print(X_pick_test)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  1.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0. -1.  0. ...  0.  0. -1.]
 [ 0.  0.  0. ...  0.  0.  1.]]


In [32]:
inter_test = np.concatenate((X_test.transpose(), X_pick_test.transpose()))
X_new_test = inter_test.transpose()

In [33]:
print(clf.predict_proba(X_new_test))

[[0.12789953 0.87210047]
 [0.15693965 0.84306035]
 [0.79334359 0.20665641]
 ...
 [0.72653589 0.27346411]
 [0.29808473 0.70191527]
 [0.4715741  0.5284259 ]]


In [38]:
probs = []
for k in clf.predict_proba(X_new_test):
    probs.append(k[0])
#print(probs.max, probs.min())
np_probs = np.array(probs)
print(np.max(np_probs), np.min(np_probs))

0.9922287194875344 0.0009406101509025211


In [35]:
a = pandas.read_csv('features_test.csv')
print(a)
a['radiant_win'] = probs
answer = a[['match_id', 'radiant_win']]
print(answer)
answer.to_csv('my_csv_export.csv', index = False)

       match_id  start_time  lobby_type  r1_hero  r1_level  r1_xp  r1_gold  \
0             6  1430287923           0       93         4   1103     1089   
1             7  1430293357           1       20         2    556      570   
2            10  1430301774           1      112         2    751      808   
3            13  1430323933           1       27         3    708      903   
4            16  1430331112           1       39         4   1259      661   
...         ...         ...         ...      ...       ...    ...      ...   
17172    114369  1450212780           7       11         5   2054     1941   
17173    114377  1450222875           1        3         3    748      605   
17174    114378  1450223593           1       85         2    575      499   
17175    114393  1450244771           0        7         4   1844     1176   
17176    114398  1450255429           1        8         4   1215     1305   

       r1_lh  r1_kills  r1_deaths  ...  radiant_ward_sentry_cou

In [36]:
check = pandas.read_csv('my_csv_export.csv')
print(check)

       match_id  radiant_win
0             6     0.127900
1             7     0.156940
2            10     0.793344
3            13     0.081746
4            16     0.725002
...         ...          ...
17172    114369     0.143768
17173    114377     0.264100
17174    114378     0.726536
17175    114393     0.298085
17176    114398     0.471574

[17177 rows x 2 columns]
